# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#import csv
city_data = "../WeatherPy/output_data/cities.csv"
city_data_df = pd.read_csv(city_data)
city_df = pd.DataFrame(city_data_df)
city_df

,city,latitude,longitude,max temp (F),humidity,cloudiness,windspeed,country,date
0,Benalla,-36.55,145.98,48.99,70,40,11.41,AU,1596503427
1,Atuona,-9.80,-139.03,79.36,72,0,13.44,PF,1596503427
2,Jamestown,42.10,-79.24,66.00,93,90,1.54,US,1596503427
3,Wuhu,31.34,118.38,84.20,79,75,8.95,CN,1596503427
4,Bonavista,48.65,-53.11,68.00,72,43,15.01,CA,1596503313
...,...,...,...,...,...,...,...,...,...
541,Muros,42.78,-9.06,66.00,84,30,7.18,ES,1596503560
542,At-Bashi,41.17,75.81,56.08,50,0,0.47,KG,1596503560
543,Manaus,-3.10,-60.02,82.40,69,19,3.36,BR,1596503560
544,Natitingou,10.30,1.38,73.40,94,40,6.93,BJ,1596503560


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

locations = city_df[["latitude", "longitude"]]
weights = city_df["humidity"]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

# Save the figure
#plt.savefig("output_data/cities_heatmap.png")

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Narrow down city list by conditions
ideal_df1 = city_df.loc[(city_df["max temp (F)"] < 90)]
ideal_df2 = ideal_df1.loc[(city_df["max temp (F)"] > 80)]
ideal_df3 = ideal_df2.loc[(city_df["humidity"] < 60)] 
ideal_df4 = ideal_df3.loc[(city_df["windspeed"] < 10)]
ideal_df5 = ideal_df4.loc[(city_df["cloudiness"] < 10)]
ideal_df = ideal_df5.dropna()
ideal_df

,city,latitude,longitude,max temp (F),humidity,cloudiness,windspeed,country,date
79,Maningrida,-12.07,134.27,81.19,51,0,6.73,AU,1596503146
186,Tezu,27.92,96.17,87.60,56,0,1.07,IN,1596503471
216,Pullman,46.73,-117.18,82.00,26,1,9.17,US,1596503219
238,Newberg,45.30,-122.97,84.99,35,1,3.20,US,1596503484
360,Riyadh,24.69,46.72,87.80,27,0,3.60,SA,1596503226
520,Mangan,27.52,88.53,84.22,44,4,1.03,IN,1596503555
524,Dickinson,29.46,-95.05,89.01,58,1,6.93,US,1596503556
540,Umluj,25.02,37.27,84.18,48,0,5.37,SA,1596503559
545,Batesville,35.77,-91.64,80.60,57,1,4.70,US,1596503340


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Set up column to hold new info
ideal_df["hotel_name"] = ""
ideal_df

,city,latitude,longitude,max temp (F),humidity,cloudiness,windspeed,country,date,hotel_name
79,Maningrida,-12.07,134.27,81.19,51,0,6.73,AU,1596503146,
186,Tezu,27.92,96.17,87.60,56,0,1.07,IN,1596503471,
216,Pullman,46.73,-117.18,82.00,26,1,9.17,US,1596503219,
238,Newberg,45.30,-122.97,84.99,35,1,3.20,US,1596503484,
360,Riyadh,24.69,46.72,87.80,27,0,3.60,SA,1596503226,
520,Mangan,27.52,88.53,84.22,44,4,1.03,IN,1596503555,
524,Dickinson,29.46,-95.05,89.01,58,1,6.93,US,1596503556,
540,Umluj,25.02,37.27,84.18,48,0,5.37,SA,1596503559,
545,Batesville,35.77,-91.64,80.60,57,1,4.70,US,1596503340,


In [6]:
# Use lat, lng to use places API to find nearest hotel

hotel_name = []

# use iterrows to iterate through pandas dataframe
for index, row in ideal_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    lat = row["latitude"]
    lng = row["longitude"]
    city = ideal_df["city"]

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types": "lodging",
        "key": g_key
    }
    
    
    #get city name from df
    city_name = row["city"]
    
    params["keyword"] = hotel_name
    
    # Run request
    (f"Retrieving Results for index {index}: {city}")
    response = requests.get(base_url, params=params).json()


    # print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
      #print(response.url)
    results = response["results"]

    #print(json.dumps(hotels, indent=4, sort_keys=True))

    try:
        print(f"Closest hotel to {city_name} is {results[0]['name']}.")
        ideal_df.loc[index, "hotel_name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

    print("------------")
    
# 

   

Closest hotel to Maningrida is Dhukurrdji Lodge.
------------
Closest hotel to Tezu is Circuit House.
------------
Closest hotel to Pullman is Holiday Inn Express & Suites Pullman.
------------
Closest hotel to Newberg is The Allison Inn & Spa.
------------
Closest hotel to Riyadh is InterContinental Riyadh.
------------
Closest hotel to Mangan is Dzongu Lee, Home Stay.
------------
Closest hotel to Dickinson is Candlewood Suites League City.
------------
Closest hotel to Umluj is Moon light Furnished Units.
------------
Closest hotel to Batesville is Holiday Inn Express & Suites Batesville.
------------


In [7]:
ideal_df

,city,latitude,longitude,max temp (F),humidity,cloudiness,windspeed,country,date,hotel_name
79,Maningrida,-12.07,134.27,81.19,51,0,6.73,AU,1596503146,Dhukurrdji Lodge
186,Tezu,27.92,96.17,87.60,56,0,1.07,IN,1596503471,Circuit House
216,Pullman,46.73,-117.18,82.00,26,1,9.17,US,1596503219,Holiday Inn Express & Suites Pullman
238,Newberg,45.30,-122.97,84.99,35,1,3.20,US,1596503484,The Allison Inn & Spa
360,Riyadh,24.69,46.72,87.80,27,0,3.60,SA,1596503226,InterContinental Riyadh
520,Mangan,27.52,88.53,84.22,44,4,1.03,IN,1596503555,"Dzongu Lee, Home Stay"
524,Dickinson,29.46,-95.05,89.01,58,1,6.93,US,1596503556,Candlewood Suites League City
540,Umluj,25.02,37.27,84.18,48,0,5.37,SA,1596503559,Moon light Furnished Units
545,Batesville,35.77,-91.64,80.60,57,1,4.70,US,1596503340,Holiday Inn Express & Suites Batesville


In [8]:
hotel_df = ideal_df.rename(columns={"city": "City", "latitude": "Lat", "longitude": "Lng", 
                                    "max temp(F)": "Max Temp (F)", "humidity": "Humidity", "cloudiness": "Cloudiness (%)",
                                    "windspeed": "Wind Speed (mph)", "country": "Country", "date": "Date",
                                    "hotel_name": "Hotel Name"})
hotel_df

,City,Lat,Lng,max temp (F),Humidity,Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
79,Maningrida,-12.07,134.27,81.19,51,0,6.73,AU,1596503146,Dhukurrdji Lodge
186,Tezu,27.92,96.17,87.60,56,0,1.07,IN,1596503471,Circuit House
216,Pullman,46.73,-117.18,82.00,26,1,9.17,US,1596503219,Holiday Inn Express & Suites Pullman
238,Newberg,45.30,-122.97,84.99,35,1,3.20,US,1596503484,The Allison Inn & Spa
360,Riyadh,24.69,46.72,87.80,27,0,3.60,SA,1596503226,InterContinental Riyadh
520,Mangan,27.52,88.53,84.22,44,4,1.03,IN,1596503555,"Dzongu Lee, Home Stay"
524,Dickinson,29.46,-95.05,89.01,58,1,6.93,US,1596503556,Candlewood Suites League City
540,Umluj,25.02,37.27,84.18,48,0,5.37,SA,1596503559,Moon light Furnished Units
545,Batesville,35.77,-91.64,80.60,57,1,4.70,US,1596503340,Holiday Inn Express & Suites Batesville


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)


# # fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

# Display figure



Figure(layout=FigureLayout(height='420px'))

In [11]:
# Save the figure
plt.savefig("output_data/ideal_marker_heatmap.png")

<Figure size 432x288 with 0 Axes>